In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


train = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
test = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')
submission = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')

#### 提出形式を確認する
Id(写真)に紐付いたPawpularity(魅力度)を予測する。

In [ ]:
submission

#### データ概要を把握する

In [ ]:
import pandas_profiling
# train.profile_report()
train.iloc[0]

#### カラムを理解する(DeepL翻訳)
各ペットの写真には，以下の特徴のそれぞれについて，1（Yes）または0（No）の値が付けられてい流。
* Focus - ペットは、整然とした背景の中で、近すぎず遠すぎず、際立っている。
* Eyes - 両目が正面またはそれに近い方向を向いており、少なくとも1つの目/瞳がきちんとクリアになっている。
* Face - 正面またはそれに近い方向を向いている、はっきりとした顔。
* Bear - 1匹のペットが写真のかなりの部分を占めている（おおよそ写真の幅または高さの50％以上）。
* Action - ペットが何かアクションを起こしている最中（例：ジャンプ）。
* Accessory - 首輪やリードを除く、付随する物理的またはデジタル的なアクセサリー/小道具（例：おもちゃ、デジタルステッカー）。
* Group - 1匹以上のペットが写っていること。
* Collage - デジタル的にレタッチされた写真（デジタルフォトフレームを使用したもの、複数の写真を組み合わせたものなど）。
* Human - 写真に写っている人間。
* Occlusion - 特定の望ましくないオブジェクトがペットの一部を遮っている状態（例：人間、ケージ、フェンス）。すべての遮蔽物がオクルージョンとは限らない。
* Info - カスタムで追加されたテキストやラベル（例：ペットの名前、説明）。
* Blur - 特にペットの目や顔など、目立ったピンボケやノイズがある状態。ブラーのエントリーでは、「目」列は常に0に設定されます。

#### 欠損値に関して
train、testともに欠損値は存在しないので補完作業は不要

In [ ]:
print(train.isnull().sum(),'\n------------\n',
        test.isnull().sum())

#### 魅力度1のデータと100のデータを確認してみる
※魅力度0は存在しない

In [ ]:
train[train['Pawpularity']==1]

In [ ]:
train[train['Pawpularity']==100]

In [ ]:
import matplotlib.pyplot as plt
import cv2

train_path = "../input/petfinder-pawpularity-score/train"
# 最初のIDの写真を確認
path = os.path.join(train_path,train["Id"].iloc[0]+".jpg")

In [ ]:
img = cv2.imread(path)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)

## 仮説
#### Pawpularity(魅力度)が低い写真の特徴
1. Accessory, Human, Occlusion, Blurのデータが1の時
    * つまり、余分な情報、ノイズが多くある場合
2. 上記カラム以外の値が0の方が多い時
    * 特徴が少ないので魅力度が減少する

#### Pawpularity(魅力度)が高い写真の特徴
1. Accessory, Human, Occlusion, Blurのデータが0の時
    * 余分な情報、ノイズが少ない場合
2. 上記カラム以外の値が１の方が多い時
    * 特徴が多いので魅力度が増加する

In [ ]:
lists = ['Accessory', 'Human', 'Occlusion', 'Blur']
tmp_train = train.copy()
# 魅力度50以下の写真の平均値を求める
low = tmp_train[tmp_train['Pawpularity']<=50]
# 魅力度50以上の写真の平均値を求める
high = tmp_train[tmp_train['Pawpularity']>50]

#### 1の仮説

In [ ]:
for l in lists:
    print('【low】 {} : {}'.format(l,low[l].mean()))
    print('【high】 {} : {}'.format(l,high[l].mean()))
    if low[l].mean() > high[l].mean():
        print('lowの方が高い')
    else:
        print('highの方が高い')
    print('---------------------------')

* 仮説の1番目に関して、あまり有意差は見られなかった。

#### 2の仮説

In [ ]:
tmp_train.drop(lists, axis=1, inplace=True)
# Id, Pawpularity以外の8カラム
for i in range(1, 9):
    print('【low】 {} : {}'.format(low.columns[i],low.iloc[:, i].mean()))
    print('【high】 {} : {}'.format(high.columns[i],high.iloc[:, i].mean()))
    if low.iloc[:, i].mean() > high.iloc[:, i].mean():
        print('lowの方が高い')
    else:
        print('highの方が高い')
    print('---------------------------')

* 仮説の2番目に関して、あまり有意差は見られなかった

以上より仮説は特に重要視する点ではない。

## LGBMでsubmitしてみる

In [ ]:
train_id = train['Id']
test_id = test['Id']

y_train = train['Pawpularity']
X_train = train.drop(['Id','Pawpularity'], axis=1)
X_test = test.drop(['Id'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = \
    train_test_split(X_train, y_train, test_size=0.3, shuffle=True, random_state=0)

In [ ]:
X_test.dtypes

#### 学習、予測

In [ ]:
# データは訓練用とテスト用の2種類を登録する
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

In [ ]:
params = {
    'metric': 'rmse'
}

model = lgb.train(params, lgb_train,
                 valid_sets=[lgb_train, lgb_eval],
                 verbose_eval=10,
                 num_boost_round=1000,
                 early_stopping_rounds=10)
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
y_pred[:10]

In [ ]:
sub = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')

sub['Pawpularity'] = y_pred
sub.to_csv('submission.csv', index=False)

In [ ]:
sub